# Block Generator

This notebook generates a list of blocks for the novel object eperiments, given the number of objects `N_ITEMS` and the number of novels we'd like to display in a trial `N_ITEMS_PER_BLOCK`.

The generated blocks meet the following condition:

1. Each unique pair of objects (obj1, obj2) will at least appear in one block.
2. We run mutliple simulations trying to minimize the number of total trials (blocks), in order to reduce the effort required from subjects to complete the experiment.

This is similar to find a [Steiner System](https://en.wikipedia.org/wiki/Steiner_system) of $S(2,k,n)$, except that we do not guarantee the opitimal solution, i.e. the minumum number of blocks.

- $k$: number of objects per block
- $n$: number of items

The algorithm is as follows:

<center>
<img src="block_generation_algorithm.png" alt="Block generation algorithm" width="600" height=auto>
</center>


Latex code for generating the pseudocode above:

```Latex
\documentclass{article}
\usepackage{algorithm}
\usepackage{algpseudocode}
\begin{document}

\begin{algorithm}
    \caption{Algorithm for Generating Blocks}\label{alg:block}
    \begin{algorithmic}[1]
        \Require $N > 0$
        \Require $K > 0$
        \vspace{\baselineskip}

        \State $\mathcal{P} \gets$ all possible pairs of numbers from $1$ to $N$
        \State $\mathcal{P}^* \gets$ empty set
        \State $\mathcal{B} \gets$ empty set
        \vspace{\baselineskip}

        \While{$\mathcal{P}$ is not empty}
            \State $b \gets$ empty set
            \While{$|b| < K$ and $\mathcal{P}$ is not empty}  \Comment{More objects are needed}
                \State $p \gets$ random pair from $\mathcal{P}$
                \If{$p$ is not in $\mathcal{P}^*$}
                    \State add $p_0$ to $b$
                    \If{$|b| < K$}
                        \State add $p_1$ to $b$
                         \State $\mathcal{P} \gets \mathcal{P} - \{p\}$ \Comment{Pair consumed}
                    \EndIf
                    \vspace{\baselineskip}
                    \State $\mathcal{P}^* \gets \mathcal{P}^* \cup$ all possible pairs of numbers from $b$
                    \State $\mathcal{P} \gets \mathcal{P} - \mathcal{P}^*$  \Comment{Remove the covered pairs}
                \EndIf
            \EndWhile
            \vspace{\baselineskip}
            \If{$b$ is not empty}
                \State $\mathcal{B} \gets \mathcal{B} \cup b$
            \EndIf
        \EndWhile
        \vspace{\baselineskip}

        \State \textbf{return} $\mathcal{B}$
    \end{algorithmic}
\end{algorithm}

\end{document}

```


In [1]:
from itertools import combinations
from collections import deque
from random import shuffle
from tqdm.autonotebook import tqdm

import pandas as pd


<ipython-input-1-ee3dce4abcd2>:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
N_ITEMS = 30
N_ITEMS_PER_BLOCK = 16


In [3]:
# Generate all possible combinations of (x, y) from 1 to N_ITEMS
PAIRS = list(combinations(range(1, N_ITEMS + 1), 2))
print("number of pairs:", len(PAIRS))
print("exmple pair:", PAIRS[0])


number of pairs: 435
exmple pair: (1, 2)


In [4]:
def generate_blocks(n_items, n_items_per_block):
    # Generate all possible combinations of (x, y) from 1 to n_items
    pairs = deque(combinations(range(1, n_items + 1), 2))

    blocks = []
    pair_used = set()
    while pairs:
        block = set()
        # randomize the remaining pairs
        shuffle(pairs)
        while pairs and len(block) < n_items_per_block:
            pair = pairs[0]
            if pair not in pair_used:
                block.add(pair[0])

                if len(block) < n_items_per_block:
                    block.add(pair[1])
                    pairs.popleft()  # remove the pair from the list

                # For all possible pair the current block can cover, mark then as used
                for used_pair in combinations(block, 2):
                    pair_used.add(used_pair)

                pairs = deque(set(pairs) - pair_used)

        # Save this block if it is not empty
        if block:
            blocks.append(sorted(block))

    return blocks


In [5]:
test_blocks = generate_blocks(N_ITEMS, N_ITEMS_PER_BLOCK)
len(test_blocks)


8

In [6]:
def validate_blocks(_blocks, n_items, n_items_per_block):
    """Make sure the block we created is valid"""
    for block in _blocks:
        assert len(block) <= n_items_per_block

    pairs = set(combinations(range(1, n_items + 1), 2))
    for i, j in pairs:
        found = False
        for block in _blocks:
            if i in block and j in block:
                found = True
                break
        if not found:
            print(i, j)
            return
    print("Block is Valid")


In [7]:
validate_blocks(test_blocks, N_ITEMS, N_ITEMS_PER_BLOCK)


Block is Valid


In [8]:
def get_minimum_blocks(n_items, n_items_per_block, n_trials=500):
    """
    Run `generate_blocks` for `n_trials` times and return the one with the minimum number of blocks
    """
    best_block = []
    min_blocks = 9999

    for _ in range(n_trials):
        _blocks = generate_blocks(n_items, n_items_per_block)
        if len(_blocks) < min_blocks:
            min_blocks = len(_blocks)
            best_block = _blocks

    return best_block


In [9]:
# Try different number of items per block, find a reasonable number for my experiment
block_size_and_min_trials = []

for block_size in tqdm(range(8, 20)):
    blocks = get_minimum_blocks(N_ITEMS, block_size, n_trials=1000)
    block_size_and_min_trials.append((block_size, len(blocks)))

pd.DataFrame(block_size_and_min_trials, columns=["block_size", "min_trials"])


  0%|          | 0/12 [00:00<?, ?it/s]

,block_size,min_trials
0,8,29
1,9,23
2,10,19
3,11,15
4,12,13
5,13,10
6,14,9
7,15,7
8,16,6
9,17,6


In [10]:
my_blocks = get_minimum_blocks(N_ITEMS, 16, n_trials=5000)


In [11]:
validate_blocks(my_blocks, N_ITEMS, N_ITEMS_PER_BLOCK)


Block is Valid


In [12]:
my_blocks


[[3, 4, 7, 8, 9, 11, 13, 14, 15, 17, 18, 21, 22, 23, 26, 30],
 [2, 3, 5, 6, 10, 11, 12, 16, 18, 19, 20, 23, 25, 27, 28, 29],
 [1, 3, 5, 7, 9, 10, 14, 15, 16, 19, 20, 22, 24, 26, 28, 30],
 [1, 2, 4, 5, 6, 7, 8, 12, 13, 15, 17, 19, 21, 24, 27, 28],
 [1, 2, 6, 7, 9, 10, 12, 14, 18, 21, 22, 25, 26, 27, 29, 30],
 [1, 4, 8, 10, 11, 13, 15, 16, 17, 18, 20, 21, 23, 24, 25, 29]]

In [13]:
[len(x) for x in my_blocks]


[16, 16, 16, 16, 16, 16]